# Code for converting txt files into 3 word chunks

In [ ]:
with open("/content/Copy of walk_in_the_park_396wds_scanner_confirmed.txt", "r") as f:
     story = f.read()

In [ ]:
import re
import pandas as pd

def conversion(text):
    text = text.replace('\n', ' ') # Remove line breaks (data cleaning)
    text = ' '.join(text.split())  #remove multiple spaces
    words = text.split()

    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+3]
        chunk_str = ' '.join(chunk)
        chunks.append(chunk_str)
        i += 3

    return chunks

chunks = conversion(story)

# Test
for chunk in chunks[:10]:
    print(chunk)

﻿It was fall,
and I embarked
on a stroll
through an area
near my home.
I came across
many beautiful things,
and the cool
breeze was invigorating.
It's nice to


In [ ]:
# Saving to CSV
df = pd.DataFrame({'chunk': chunks})
df.to_csv("three_word_split_story4.csv", index=False)

# Embedding Extraction

In [ ]:
from huggingface_hub import login
login()  # logging in to access LLama weights

In [ ]:
!pip install transformers accelerate xformers --upgrade
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 137.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 8.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "meta-llama/Llama-2-7b-chat-hf"

# Loading tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.float16,
    device_map="auto",
    output_hidden_states=True
)
model.eval()

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['output_hidden_states']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
   

In [ ]:

df = pd.read_csv('/content/three_word_split_story4.csv')
chunks = df['chunk'].tolist()

#Testing
print(chunks[:10])

['\ufeffIt was fall,', 'and I embarked', 'on a stroll', 'through an area', 'near my home.', 'I came across', 'many beautiful things,', 'and the cool', 'breeze was invigorating.', "It's nice to"]


In [ ]:
chunks[0] = chunks[0].lstrip('\ufeff')

In [ ]:
import torch
import numpy as np

def get_llama_embedding_with_context(chunk_idx, chunks, context_size=10):
    # Building context window up to 10 previous chunks plus the current one
    context_start = max(0, chunk_idx - context_size)
    context_window = chunks[context_start:chunk_idx+1]  # include current chunk

    # Joining context window into one string
    full_input = ' '.join(context_window)

    inputs = tokenizer(full_input, return_tensors='pt')
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    hidden_states = outputs.hidden_states

    # Select middle layer according to papers
    middle_layer = hidden_states[16].squeeze(0)  # shape: [seq_len, hidden_dim]

    # Get token IDs for current chunk
    target_chunk = chunks[chunk_idx]
    target_ids = tokenizer(target_chunk, return_tensors='pt')['input_ids'].squeeze()
    target_len = target_ids.shape[0]

    # Get last N tokens of the input as target chunk
    target_embeddings = middle_layer[-target_len:]
    avg_embedding = target_embeddings.mean(dim=0)

    return avg_embedding.cpu().numpy()

In [ ]:
all_embeddings = []

for i in range(len(chunks)):
    try:
        emb = get_llama_embedding_with_context(i, chunks)
        all_embeddings.append(emb)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        all_embeddings.append(np.zeros((4096,)))

In [ ]:
import numpy as np

np.save('llama_embeddings_story4.npy', np.array(all_embeddings))